# Setup


In [44]:
%cd ..
!eza --icons

/data/cmi/notes/sem-4/applied-ml/assignments/assignment-2
 data   images   README.md   src


In [2]:
!dvc init -f

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [4]:
!git status
!git commit -m "🎉 init: initialised dvc"

On branch main

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   .dvc/.gitignore
	new file:   .dvc/config
	new file:   .dvcignore

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	README.md
	data/
	images/
	src/



[main (root-commit) abf5167] 🎉 init: initialised dvc
 3 files changed, 6 insertions(+)
 create mode 100755 .dvc/.gitignore
 create mode 100755 .dvc/config
 create mode 100755 .dvcignore


## Decoupling storage and compute

I added google drive as remote storage for dvc.


In [62]:
!dvc remote add -d remotestorage gdrive://1sK3RLupgUrr1YKd3A8BOjpwg5AAVqV3p -f

Setting 'remotestorage' as a default remote.


In [7]:
!git add .dvc/config
!git status
!git commit -m "⚡ update: updated dvc config"

On branch main
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   .dvc/config

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	README.md
	data/
	images/
	src/

[main 0bd0ae0] ⚡ update: updated dvc config
 1 file changed, 4 insertions(+)


In [45]:
%cd src/

/data/cmi/notes/sem-4/applied-ml/assignments/assignment-2/src


# Imports


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tabulate import tabulate

In [2]:
import warnings

warnings.simplefilter("ignore")

# Loading the dataset and splitting it


I use the following code to clean the `emails.csv` and save it as `raw_data.csv` . This code is from my assignment 1 [ `prepare.ipynb` ](https://github.com/Roudranil/applied-ml-assignment-1/blob/main/src/prepare.ipynb)

```python
def startstrip(text: str, subtext: str):
    if text.startswith(subtext):
        text = text[len(subtext) :]
    return text

def clean_text(text: str):
    # convert text to lower case to make it easier to do the preprocessing
    text = text.lower()

    # remove the "subject :" from the beginning
    text = startstrip(text, "subject: ")
    text = text.lstrip("subject: ")

    # finding if the mail was a reply to another mail
    is_reply = 1 if text.startswith("re :") else 0
    text = startstrip(text, "re : ")

    # cleaning the text
    text = re.sub(r"\d+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = re.sub(r"[^\x00-\x7F]+", "", text)

    stopwords_list = stopwords.words("english")
    text = " ".join(word for word in text.split() if word not in stopwords_list)

    return text, is_reply

cleaned_data = pd.DataFrame(
    data["text"].apply(lambda x: clean_text(x)).tolist(), columns=["text", "is_reply"]
)
cleaned_data["spam"] = data["spam"]

cleaned_data.to_csv("../data/raw_data.csv", index=False)
```


In [13]:
preprocessed_raw_data = pd.read_csv("../data/raw_data.csv")
# Split the data into train/validation/test with the specified random seeds
train, test = train_test_split(preprocessed_raw_data, test_size=0.2, random_state=42)
train, val = train_test_split(
    train, test_size=0.25, random_state=42
)  # 0.25 x 0.8 = 0.2

In [15]:
# Save the splits in the 'data' folder
train.to_csv("../data/train.csv", index=False)
val.to_csv("../data/validation.csv", index=False)
test.to_csv("../data/test.csv", index=False)

### Tracking dataset versions with dvc


In [46]:
%cd ../data
!eza --icons

/data/cmi/notes/sem-4/applied-ml/assignments/assignment-2/data
 raw_data.csv   test.csv   train.csv   validation.csv


In [47]:
!dvc add raw_data.csv test.csv train.csv validation.csv

⠋ Checking graph                                       core>
  0% Adding...|                       | raw_data.csv |0/4 [00:00<?,     ?file/s]
!
                                                                                
!
  0% Checking cache in '/data/cmi/notes/sem-4/applied-ml/assignments/assignment-
                                                                                
!
  0%|          |Adding raw_data.csv to cache          0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /data/cmi/notes/sem-4/app0/1 [00:00<?,    ?files/s]
 25% Adding...|█████▊                 | test.csv |1/4 [00:00<00:00,  7.31file/s]
!
                                                                                
!
  0% Checking cache in '/data/cmi/notes/sem-4/applied-ml/assignments/assignment-
                                                                                
!
  0%|          |Adding test.csv to

In [53]:
!git status
!git add train.csv.dvc validation.csv.dvc test.csv.dvc raw_data.csv.dvc .gitignore
!git commit -m "⚡ update: enabled versioning data with dvc"

On branch main
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../README.md
	./
	../images/
	../src/

nothing added to commit but untracked files present (use "git add" to track)


[main 182e994] ⚡ update: enabled versioning data with dvc
 5 files changed, 28 insertions(+)
 create mode 100644 data/.gitignore
 create mode 100644 data/raw_data.csv.dvc
 create mode 100644 data/test.csv.dvc
 create mode 100644 data/train.csv.dvc
 create mode 100644 data/validation.csv.dvc


In [65]:
!dvc push

Pushing
!
  0% Checking cache in '1sK3RLupgUrr1YKd3A8BOjpwg5AAVqV3p/files/md5'| |0/? [00:0
  0% Querying cache in '1sK3RLupgUrr1YKd3A8BOjpwg5AAVqV3p/files/md5'| |1/256 [00
                                                                                
!
  0% Checking cache in '/data/cmi/notes/sem-4/applied-ml/assignments/assignment-
                                                                                
!
  0%|          |Pushing to gdrive                     0/3 [00:00<?,     ?file/s]
  0%|          |Pushing to gdrive                     0/3 [00:00<?,     ?file/s]

!

  0%|          |/data/cmi/notes/sem-4/applied-m0.00/1.09M [00:00<?,        ?B/s]

  1%|          |/data/cmi/notes/sem-4/appl8.00k/1.09M [00:02<04:45,    3.96kB/s]

                                                                                
 33%|███▎      |Pushing to gdrive                 1/3 [00:04<00:08,  4.08s/file]

!

  0%|          |/data/cmi/notes/sem-4/applied-m0.00/3.15M [00:00<?,        ?B/s]


!


### Updating splits with new random seed


In [42]:
preprocessed_raw_data = pd.read_csv("../data/raw_data.csv")
# Split the data into train/validation/test with the specified random seeds
train, test = train_test_split(preprocessed_raw_data, test_size=0.2, random_state=1024)
train, val = train_test_split(
    train, test_size=0.25, random_state=1024
)  # 0.25 x 0.8 = 0.2
# Save the splits in the 'data' folder
train.to_csv("../data/train.csv", index=False)
val.to_csv("../data/validation.csv", index=False)
test.to_csv("../data/test.csv", index=False)

In [48]:
%cd ../data
!eza

/data/cmi/notes/sem-4/applied-ml/assignments/assignment-2/data
raw_data.csv      test.csv      train.csv      validation.csv
raw_data.csv.dvc  test.csv.dvc  train.csv.dvc  validation.csv.dvc


In [49]:
# Add the updated datasets to DVC
!dvc add train.csv validation.csv test.csv

⠋ Checking graph                                       core>
  0% Adding...|                          | train.csv |0/3 [00:00<?,     ?file/s]
!
                                                                                
!
  0% Checking cache in '/data/cmi/notes/sem-4/applied-ml/assignments/assignment-
                                                                                
!
  0%|          |Adding train.csv to cache             0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /data/cmi/notes/sem-4/app0/1 [00:00<?,    ?files/s]
  0% Adding...|                     | validation.csv |0/3 [00:00<?,     ?file/s]
!
                                                                                
!
  0% Checking cache in '/data/cmi/notes/sem-4/applied-ml/assignments/assignment-
                                                                                
!
  0%|          |Adding validation.

In [51]:
!git add test.csv.dvc train.csv.dvc validation.csv.dvc
!git status
!git commit -m "⚡ update: updated data splits with new random seed (1024)"

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   test.csv.dvc
	modified:   train.csv.dvc
	modified:   validation.csv.dvc

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../src/prepare.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../README.md
	../images/

[main a205efc] ⚡ update: updated data splits with new random seed (1024)
 3 files changed, 6 insertions(+), 9 deletions(-)


In [52]:
!dvc push

Pushing
!
  0% Checking cache in '1sK3RLupgUrr1YKd3A8BOjpwg5AAVqV3p/files/md5'| |0/? [00:0
  0% Querying cache in '1sK3RLupgUrr1YKd3A8BOjpwg5AAVqV3p/files/md5'| |1/256 [00
                                                                                
!
  0% Checking cache in '/data/cmi/notes/sem-4/applied-ml/assignments/assignment-
                                                                                
!
  0%|          |Pushing to gdrive                     0/3 [00:00<?,     ?file/s]
  0%|          |Pushing to gdrive                     0/3 [00:00<?,     ?file/s]

!

  0%|          |/data/cmi/notes/sem-4/applied-m0.00/1.16M [00:00<?,        ?B/s]

  1%|          |/data/cmi/notes/sem-4/appl8.00k/1.16M [00:03<08:02,    2.50kB/s]

 15%|█▍        |/data/cmi/notes/sem-4/appli176k/1.16M [00:03<00:15,    66.7kB/s]

 23%|██▎       |/data/cmi/notes/sem-4/appli272k/1.16M [00:03<00:09,     102kB/s]

 32%|███▏      |/data/cmi/notes/sem-4/appli376k/1.16M [00:04<00:05,     139kB/s]

 40%|

# Checking out original data split and printing distribution


In [53]:
!git log

commit a205efc6eec4918143f4b17cd458acb03eaae28e (HEAD -> main)
Author: Roudranil <nil0auditore78@gmail.com>
Date:   Tue Feb 20 12:06:20 2024 +0530

    ⚡ update: updated data splits with new random seed (1024)

commit a1996cb1e2704e8bc4e7dcf8fe9b2cd79264834f (origin/main)
Author: Roudranil <nil0auditore78@gmail.com>
Date:   Mon Feb 19 12:53:58 2024 +0530

    🔧 code: prepare.ipynb updated to track data with dvc

commit 182e994a039e751cb50352ea0256d2889eafbd2f
Author: Roudranil <nil0auditore78@gmail.com>
Date:   Mon Feb 19 12:51:42 2024 +0530

    ⚡ update: enabled versioning data with dvc

commit 0bd0ae00c95c795a296dd9111044a26e5d114120
Author: Roudranil <nil0auditore78@gmail.com>
Date:   Mon Feb 19 12:25:41 2024 +0530

    ⚡ update: updated dvc config

commit abf516717acc2c8aa10571234df543217d0414db
Author: Roudranil <nil0auditore78@gmail.com>
Date:   Mon Feb 19 11:57:30 2024 +0530

    🎉 init: initialised dvc


Previous version is this one

```bash
commit 182e994a039e751cb50352ea0256d2889eafbd2f
Author: Roudranil <nil0auditore78@gmail.com>
Date:   Mon Feb 19 12:51:42 2024 +0530

    ⚡ update: enabled versioning data with dvc
```


In [54]:
!git checkout 182e994a039e751cb50352ea0256d2889eafbd2f train.csv.dvc validation.csv.dvc test.csv.dvc

Updated 3 paths from 9f3aec3


In [56]:
!dvc checkout

Building workspace index                              |5.00 [00:00,  491entry/s]
Comparing indexes                                    |6.00 [00:00, 2.20kentry/s]
Applying changes                                      |3.00 [00:00,  53.0file/s]
M       validation.csv
M       train.csv
M       raw_data.csv
M       test.csv


In [57]:
def show_distribution(train, val, test):
    for dataset, name in zip([train, val, test], ["Train", "Validation", "Test"]):
        print(f"==>  {name}")
        _ = pd.DataFrame(dataset["spam"].value_counts()).reset_index()
        _["percentage"] = (_["count"] / _["count"].sum()).apply(
            lambda x: f"{x*100:.2f} %"
        )
        _.loc[-1] = [f"{'-'*6}\nTotal", f"{'-'*7}\n{_['count'].sum()}", f"{'-'*12}"]
        _.index = _.index + 1
        print(
            tabulate(
                _,
                headers=["Spam", "Count", "Percentage"],
                showindex=False,
                numalign="left",
                stralign="left",
            )
        )
        print("\n")

In [58]:
%cd ../src
!eza
train = pd.read_csv("../data/train.csv")
val = pd.read_csv("../data/validation.csv")
test = pd.read_csv("../data/test.csv")
show_distribution(train, val, test)

/data/cmi/notes/sem-4/applied-ml/assignments/assignment-2/src
prepare.ipynb
==>  Train
Spam    Count    Percentage
------  -------  ------------
0       2619     76.22 %
1       817      23.78 %
------  -------  ------------
Total   3436


==>  Validation
Spam    Count    Percentage
------  -------  ------------
0       885      77.23 %
1       261      22.77 %
------  -------  ------------
Total   1146


==>  Test
Spam    Count    Percentage
------  -------  ------------
0       856      74.69 %
1       290      25.31 %
------  -------  ------------
Total   1146




# Checking out updated data split and printing distribution


Updated data split is this one

```bash
commit a205efc6eec4918143f4b17cd458acb03eaae28e (HEAD -> main)
Author: Roudranil <nil0auditore78@gmail.com>
Date:   Tue Feb 20 12:06:20 2024 +0530

    ⚡ update: updated data splits with new random seed (1024)
```


In [60]:
%cd ../data
!eza
!git checkout a205efc6eec4918143f4b17cd458acb03eaae28e train.csv.dvc validation.csv.dvc test.csv.dvc

/data/cmi/notes/sem-4/applied-ml/assignments/assignment-2/data
raw_data.csv      test.csv      train.csv      validation.csv
raw_data.csv.dvc  test.csv.dvc  train.csv.dvc  validation.csv.dvc


Updated 3 paths from 2553738


In [61]:
!dvc checkout

Building workspace index                              |5.00 [00:00,  533entry/s]
Comparing indexes                                    |6.00 [00:00, 2.27kentry/s]
Applying changes                                      |3.00 [00:00,  61.5file/s]
M       test.csv
M       validation.csv
M       raw_data.csv
M       train.csv


In [62]:
%cd ../src
!eza
train = pd.read_csv("../data/train.csv")
val = pd.read_csv("../data/validation.csv")
test = pd.read_csv("../data/test.csv")
show_distribution(train, val, test)

/data/cmi/notes/sem-4/applied-ml/assignments/assignment-2/src
prepare.ipynb
==>  Train
Spam    Count    Percentage
------  -------  ------------
0       2597     75.58 %
1       839      24.42 %
------  -------  ------------
Total   3436


==>  Validation
Spam    Count    Percentage
------  -------  ------------
0       882      76.96 %
1       264      23.04 %
------  -------  ------------
Total   1146


==>  Test
Spam    Count    Percentage
------  -------  ------------
0       881      76.88 %
1       265      23.12 %
------  -------  ------------
Total   1146


